In [1]:
!pip uninstall -y telegram python-telegram-bot telebot pyTelegramBotAPI
!pip install python-telegram-bot==20.6
!pip install openai-whisper
!pip install torch torchvision torchaudio
!pip install librosa soundfile


Found existing installation: python-telegram-bot 13.15
Uninstalling python-telegram-bot-13.15:
  Successfully uninstalled python-telegram-bot-13.15
  Using cached python_telegram_bot-20.6-py3-none-any.whl.metadata (15 kB)
Using cached python_telegram_bot-20.6-py3-none-any.whl (548 kB)


In [2]:
import tensorflow as tf
import pickle
from keras.preprocessing.sequence import pad_sequences

# Load your saved LSTM model
model = tf.keras.models.load_model("lstm_model.keras")

# Load tokenizer
import pickle
with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

# Must match your training configuration
MAX_LEN = 100  # (use same value used during training)

# Prediction function
def predict_toxicity(text):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=MAX_LEN)
    pred = model.predict(padded)[0][0]
    return "Toxic 😡" if pred > 0.5 else "Non-toxic 😇"



In [3]:
import whisper
from pydub import AudioSegment

# Load Whisper model (choose: tiny, base, small, medium, large)
whisper_model = whisper.load_model("base.en")

def transcribe_voice(ogg_path):
    # Convert OGG to WAV
    audio = AudioSegment.from_ogg(ogg_path)
    wav_path = "converted.wav"
    audio.export(wav_path, format="wav")

    # Transcribe using Whisper
    result = whisper_model.transcribe(wav_path)
    return result["text"].strip()


In [7]:
!pip install nest_asyncio


/usr/lib/python3.12/pathlib.py:407: RuntimeWarning: coroutine 'Application.shutdown' was never awaited
  def _load_parts(self):
/usr/lib/python3.12/pathlib.py:407: RuntimeWarning: coroutine 'Application.initialize' was never awaited
  def _load_parts(self):


In [ ]:
import nest_asyncio
nest_asyncio.apply()  # Allow nested asyncio loops

from telegram import Update
from telegram.ext import ApplicationBuilder, MessageHandler, ContextTypes, filters
import os, asyncio

TOKEN = "8258212437:AAGyWHHN3NkOzUOx5GTNkAMOV7Lg-h0IAUI"

async def handle_voice(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user = update.message.from_user.first_name
    await update.message.reply_text(f"🎙️ Hey {user}, I got your voice! Transcribing...")

    # Download voice
    file = await context.bot.get_file(update.message.voice.file_id)
    await file.download_to_drive("voice.ogg")

    # Transcribe (replace with your Whisper function)
    text = transcribe_voice("voice.ogg")
    await update.message.reply_text(f"🗣 You said: {text}")

    # Predict toxicity
    result = predict_toxicity(text)
    await update.message.reply_text(f"🧠 Result: {result}")

    os.remove("voice.ogg")

# Build bot
app = ApplicationBuilder().token(TOKEN).build()
app.add_handler(MessageHandler(filters.VOICE, handle_voice))

# Run bot in Colab safely
async def main():
    await app.initialize()
    await app.start()
    await app.updater.start_polling()
    print("🤖 Bot is running...")
    await asyncio.Event().wait()

# Call coroutine directly
await main()


<frozen abc>:117: RuntimeWarning: coroutine 'Application.shutdown' was never awaited
<frozen abc>:117: RuntimeWarning: coroutine 'Application.initialize' was never awaited


🤖 Bot is running...


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 391ms/step


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


...

/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
ERROR:telegram.ext.Application:No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/telegram/ext/_application.py", line 1195, in process_update
    await coroutine
  File "/usr/local/lib/python3.12/dist-packages/telegram/ext/_basehandler.py", line 153, in handle_update
    return await self.callback(update, context)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-1139670414.py", line 23, in handle_voice
    result = predict_toxicity(text)
             ^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-2645091030.py", line 20, in predict_toxicity
    pred = model.predict(padded)[0][0]
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/keras/src/utils/traceb